In [ ]:
import nba_api
import pandas as pd
import time
import pprint

from nba_api.stats.static import players
player_dict = players.get_players() #properties: id, full_name, first_name, last_name, is_active
filename = "all_nba_players.json"
try:
    with open(filename, 'w') as json_file:
        json.dump(player_dict, json_file, indent=4)
    print(f"Successfully wrote data to {filename}")
except IOError as e:
    print(f"Error writing to file {filename}: {e}")

Successfully wrote data to all_nba_players.json
Failed for DeVaughn Akoon-Purcell: 'resultSet'
Failed for Timmy Allen: 'resultSet'
Failed for Peter Aluma: 'resultSet'
Failed for Antonio Anderson: 'resultSet'
Failed for Alex Antetokounmpo: 'resultSet'
Failed for Keith Appling: 'resultSet'
Failed for Joel Ayayi: 'resultSet'
Failed for Mark Baker: 'resultSet'


C:\Users\natha\AppData\Local\Temp\ipykernel_57636\3189965042.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(rows, ignore_index=True)


In [1]:
import json

def load_players_from_json(filename="all_nba_players.json"):
    with open(filename, 'r') as f:
        return json.load(f)

In [ ]:
from nba_api.stats.endpoints import playercareerstats
import time

from requests import ReadTimeout, ConnectionError
HEADERS = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.nba.com/',
    'Connection': 'keep-alive',
}
def fetch_career_stats():
    all_players = load_players_from_json()
    rows = []
    for p in all_players:
        retries = 3
        success = False
        while retries >0 and not success:
            try:
                start = time.time()
                career = playercareerstats.PlayerCareerStats(player_id=p["id"], per_mode36="PerGame", timeout = 30)
                print(f"Fetching data for {p['full_name']}")
                print(f"Time taken: {time.time() - start:.2f} seconds")
                df = career.get_data_frames()[0]
                df["PLAYER_ID"], df["PLAYER_NAME"] = p["id"], p["full_name"]
                rows.append(df)
                print(f"Successfully added {p['full_name']}")
                success = True
                time.sleep(1.2)  # NBA API rate limiting
            except (ReadTimeout, ConnectionError) as e:
                retries -= 1
                print(f"{type(e).__name__} for {p['full_name']}, retries left: {retries-1}")
                time.sleep(6)
            except Exception as e:
                print(f"Hard Failure on {p['full_name']}: {e}")
                break
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

career_stats_df = fetch_career_stats()
career_stats_df.to_csv("nba_player_career_stats_sample.csv", index=False)

Fetching data for Alaa Abdelnaby
Time taken: 0.08 seconds
Successfully added Alaa Abdelnaby
Fetching data for Zaid Abdul-Aziz
Time taken: 0.08 seconds
Successfully added Zaid Abdul-Aziz
Fetching data for Kareem Abdul-Jabbar
Time taken: 0.03 seconds
Successfully added Kareem Abdul-Jabbar
Fetching data for Mahmoud Abdul-Rauf
Time taken: 0.08 seconds
Successfully added Mahmoud Abdul-Rauf
Fetching data for Tariq Abdul-Wahad
Time taken: 0.03 seconds
Successfully added Tariq Abdul-Wahad
Fetching data for Shareef Abdur-Rahim
Time taken: 0.06 seconds
Successfully added Shareef Abdur-Rahim
Fetching data for Tom Abernethy
Time taken: 0.10 seconds
Successfully added Tom Abernethy
Fetching data for Forest Able
Time taken: 0.08 seconds
Successfully added Forest Able
Fetching data for John Abramovic
Time taken: 0.04 seconds
Successfully added John Abramovic
Fetching data for Alex Abrines
Time taken: 0.07 seconds
Successfully added Alex Abrines
Fetching data for Precious Achiuwa
Time taken: 0.05 seco

need to fix this since and split up requests as to not get a timeout/connection error

In [8]:
traded_df = career_stats_df.copy()

marked down if a player was traded during a season or not

In [32]:
sample_df = pd.read_csv("nba_player_career_stats_sample.csv")
target_column = "TEAM_ABBREVIATION"

has_value_down_1 = sample_df[target_column].shift(-1) == "TOT"
has_value_down_2 = sample_df[target_column].shift(-2) == "TOT"

sample_df["traded_mid_season"] = has_value_down_1 | has_value_down_2
sample_df["traded_mid_season"].value_counts()
sample_df.to_csv("nba_player_career_stats_sample.csv", index=False)

In [15]:
all_df = pd.read_csv("nba_player_stats_2000_2025.csv")
all_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'TEAM_COUNT',
       'SEASON'],
      dtype='object')

Example of nba_api in action with Giannis Antetokounmpo (uses player_career_stats + players static)

In [22]:
import nba_api
import pandas as pd
import time
import pprint

player_id = 203507  # Example: Giannis Antetokounmpo's player ID
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
from nba_api.stats.endpoints import playercareerstats
career = playercareerstats.PlayerCareerStats(per_mode36=  "PerGame", player_id=player_id)
df = career.get_data_frames()[0]
career_stats_df = load_players_from_json()
df["Player_Name"] = next(d["full_name"] for d in career_stats_df if d["id"] == player_id)
col = df.pop('Player_Name') 
df.insert(0, 'Player_Name', col)  
print(df)

              Player_Name  PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  GP  GS   MIN   FGM   FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM   FTA  FT_PCT  OREB  DREB   REB  AST  STL  BLK  TOV   PF   PTS
0   Giannis Antetokounmpo     203507   2013-14        00  1610612749               MIL        19.0  77  23  24.6   2.2   5.4   0.414   0.5   1.5    0.347  1.8   2.6   0.683   1.0   3.4   4.4  1.9  0.8  0.8  1.6  2.2   6.8
1   Giannis Antetokounmpo     203507   2014-15        00  1610612749               MIL        20.0  81  71  31.4   4.7   9.6   0.491   0.1   0.5    0.159  3.2   4.3   0.741   1.2   5.5   6.7  2.6  0.9  1.0  2.1  3.1  12.7
2   Giannis Antetokounmpo     203507   2015-16        00  1610612749               MIL        21.0  80  79  35.3   6.4  12.7   0.506   0.4   1.4    0.257  3.7   5.1   0.724   1.4   6.2   7.7  4.3  1.2  1.4  2.6  3.2  16.9
3   Giannis Antetokounmpo     203507   2016-17        00  1610612749               MIL        22.0  80  80  35.6